In [23]:
import math
import json

# Get inputs

In [72]:
with open("test_input.txt") as f:
#with open("input.txt") as f:
    inp = f.readlines()
    inp = [line.rstrip() for line in inp]
inp

['[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
 '[[[5,[2,8]],4],[5,[[9,9],0]]]',
 '[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]',
 '[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]',
 '[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]',
 '[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]',
 '[[[[5,4],[7,7]],8],[[8,3],8]]',
 '[[9,3],[[9,9],[6,[4,9]]]]',
 '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
 '[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]']

In [25]:
cleaned = [json.loads(x) for x in inp]
cleaned

[[[[0, [5, 8]], [[1, 7], [9, 6]]], [[4, [1, 2]], [[1, 4], 2]]],
 [[[5, [2, 8]], 4], [5, [[9, 9], 0]]],
 [6, [[[6, 2], [5, 6]], [[7, 6], [4, 7]]]],
 [[[6, [0, 7]], [0, 9]], [4, [9, [9, 0]]]],
 [[[7, [6, 4]], [3, [1, 3]]], [[[5, 5], 1], 9]],
 [[6, [[7, 3], [3, 2]]], [[[3, 8], [5, 7]], 4]],
 [[[[5, 4], [7, 7]], 8], [[8, 3], 8]],
 [[9, 3], [[9, 9], [6, [4, 9]]]],
 [[2, [[7, 7], 7]], [[5, 8], [[9, 3], [0, 2]]]],
 [[[[5, 2], 5], [8, [3, 7]]], [[5, [7, 5]], [4, 4]]]]

## Q1

In [30]:
def get_magnitude(num = [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]):
    """3 * left elements + 2 * right elements"""
    
    left = num[0]
    right = num[1]
    
    if isinstance(left,int) and isinstance(right,int):
        return left*3 + right*2

    elif isinstance(left,list) and isinstance(right,list):
        left = get_magnitude(left)
        right = get_magnitude(right)

    elif isinstance(left,list):
        left = get_magnitude(left)
        
    else:
        right = get_magnitude(right)
    
    return get_magnitude([left, right])

get_magnitude()

3488

In [31]:
str([[[[[9,8],1],2],3],4])

'[[[[[9, 8], 1], 2], 3], 4]'

In [32]:
def next_digit(s):
    for i, c in enumerate(s):
        if c.isdigit():
            return i
    return False

In [33]:
def replace_str_index(text,index,replacement):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

In [34]:
def explode(num):
    """Reduce """
    depth = 0
    s = str(num)
    s = s.replace(" ","")
    left_index = False
    for i,c in enumerate(s):
        if c == '[':
            depth += 1
        elif c == ']':
            depth -= 1
        elif c == ',':
            continue
        elif depth > 4 and s[i+2].isdigit():
            # TODO: change replace to handle double digit nums
            if left_index:
                n = str(int(s[left_index]) + int(c))
                s = replace_str_index(s,left_index,n)
                i = i - len(s[left_index]) + len(n)

            right_index = next_digit(s[i+3:])
            if right_index:
                right_index = i + 3 + right_index
                n = str(int(s[right_index]) + int(s[i+2]))
                s = replace_str_index(s, right_index, n)
            
            # replace exploded pair with zero
            replaced = s[:i-1] + '0' + s[i+4:]
            return json.loads(replaced)
        
        else:
            left_index = i

explode([[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]])

[[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]

In [35]:
def split(num):
    s = str(num)
    s = s.replace(" ","")
    for i, c in enumerate(s):
        j = 2
        x = c
        while c.isdigit():
            x = c
            c = s[i:i+j]
            j += 1
            
        if x.isdigit() and len(x) > 1:
            first = math.floor(int(x)/2)
            print(first)
            last = math.ceil(int(x)/2)
            s = s[:i] +'['+ str(first) +','+ str(last) +']'+ s[i+len(x):]
            
            return json.loads(s)
        
split([[[[0,7],4],[[7,8],[0,13]]],[1,1]])

6


[[[[0, 7], 4], [[7, 8], [0, [6, 7]]]], [1, 1]]

In [36]:
def action_required(num, depth=0):
    """Loop through num, check if num needs to be exploded or split"""
    
    for i in num:
        if isinstance(i,int):
            if depth > 3:
                return "explode"
            if i >= 10:
                return "split"
        elif isinstance(i,list):
            action = action_required(i, depth+1)
            if action:
                return action
        
    return False

action_required([[[[0,7],4],[[7,8],[6,0]]],[8,1]])

False

In [68]:
def snailfish(homework):
    
    current_num = homework[0]
    
    for lines in homework[1:]:
        current_num = [current_num] + [lines]
        
        action = action_required(current_num)
        while action:
            if action == "explode":
                current_num = explode(current_num)
            else:
                current_num = split(current_num)
                
            action = action_required(current_num)
            
    

    return homework

In [ ]:
test = json.loads("""[1,1]
[2,2]
[3,3]
[4,4]""")

In [39]:
get_magnitude(snailfish(test))

IndexError: list index out of range

## Q2